In [53]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from functools import reduce
import math

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
parpar_dir = os.path.abspath(os.path.join(par_dir, os.pardir))

print(par_dir)
print(parpar_dir)

if par_dir not in sys.path:
    sys.path.append(par_dir)
if parpar_dir not in sys.path:
    sys.path.append(parpar_dir)
    
from repeating_classifier_training_utils import *
    
%load_ext autoreload
%matplotlib inline
%autoreload 2
index_path = os.path.join(os.getcwd(), 'Index_Storage')

/home/cmacdonald/CNN/ResNet_Analysis
/home/cmacdonald/CNN
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
f_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_extra.h5'
f = h5py.File(f_path,'r')

In [3]:
hdf5_labels = f["labels"]
hdf5_energies = f["energies"]
hdf5_positions = f["positions"]
hdf5_angles = f["angles"]
hdf5_hit_pmt = f["hit_pmt"]
hdf5_hit_charge = f["hit_charge"]
hdf5_hit_time = f["hit_time"]
hdf5_event_hits_index = f["event_hits_index"]

In [4]:
old_idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
old_test_idxs = old_idx_file['test_idxs'].astype(int)
old_train_idxs = old_idx_file['train_idxs'].astype(int)
old_val_idxs = old_idx_file['val_idxs'].astype(int)

In [20]:
hit_pmt=np.memmap(f_path, mode="r", shape=hdf5_hit_pmt.shape,
                                        offset=hdf5_hit_pmt.id.get_offset(), dtype=hdf5_hit_pmt.dtype)
charge=np.memmap(f_path, mode="r", shape=hdf5_hit_charge.shape,
                                        offset=hdf5_hit_charge.id.get_offset(), dtype=hdf5_hit_charge.dtype)

event_hits_index=np.append(hdf5_event_hits_index, hit_pmt.shape[0]).astype(np.int64)

In [17]:
old_f_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5'
old_f = h5py.File(old_f_path,'r')
old_labels = np.array(old_f['labels'])
old_f.close()

In [22]:
idxs_over_300 = []
pbar = ProgressBar(widgets=['Filtering by Total Charge: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(hdf5_labels)-old_labels.shape[0])
pbar.start()
for i, index in enumerate(range(old_labels.shape[0], len(hdf5_labels), 1)):
    start = event_hits_index[index]
    stop = event_hits_index[index+1]
    total_charge = np.sum(charge[start:stop])
    pbar.update(i)
    if total_charge > 300:
        idxs_over_300.append(index)
pbar.finish()

Checking Total Charge: 100% [00000000000000000000000000000000000] Time: 0:04:06


In [25]:
new_good_idxs = np.random.permutation(idxs_over_300)

In [26]:
new_good_idxs.shape[0]*0.9

10130312.700000001

In [28]:
new_train_idxs = new_good_idxs[:math.ceil(new_good_idxs.shape[0]*0.9)]
new_val_idxs = new_good_idxs[math.ceil(new_good_idxs.shape[0]*0.9):]

In [30]:
full_train_idxs = np.concatenate((old_train_idxs, new_train_idxs))
full_val_idxs = np.concatenate((old_val_idxs, new_val_idxs))
full_test_idxs = old_test_idxs

In [39]:
idx_sets = [full_train_idxs, full_val_idxs, full_test_idxs]
idx_dic = {}
for i in range(len(hdf5_labels)):
    idx_dic[i] = False

In [40]:
for idx_set in idx_sets:
    pbar = ProgressBar(widgets=['Checking For Repeats: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
               ' ', ETA()], maxval=len(idx_set))
    pbar.start()
    for i, idx in enumerate(idx_set):
        pbar.update(i)
        assert not idx_dic[idx]
        idx_dic[idx] = True
    pbar.finish()

Checking For Repeats: 100% [000000000000000000000000000000000000] Time: 0:00:31
Checking For Repeats: 100% [000000000000000000000000000000000000] Time: 0:00:05
Checking For Repeats: 100% [000000000000000000000000000000000000] Time: 0:00:10


In [41]:
np.savez('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz', train_idxs=full_train_idxs, val_idxs=full_val_idxs, test_idxs=full_test_idxs)

In [96]:
from io_utils.data_handling_hitarray_train import WCH5DatasetT
from io_utils.data_handling_hitarray_val import WCH5DatasetV
from io_utils.data_handling_hitarray_test import WCH5DatasetTest

train_dset = WCH5DatasetT(['/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_extra.h5'], ['/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz'], 
                          '/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M_norm_params/IWCDmPMT_4pi_fulltank_9M_trainval_norm_params.npz', 'identity', 'identity',
                                 shuffle=True, num_datasets=1, trainval_subset=None, transforms=None, collapse_arrays=False)

val_dset = WCH5DatasetV(['/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_extra.h5'], ['/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz'], 
                        '/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M_norm_params/IWCDmPMT_4pi_fulltank_9M_trainval_norm_params.npz', 'identity', 'identity',
                                 shuffle=True, num_datasets=1, trainval_subset=None, collapse_arrays=False)

test_dset = WCH5DatasetTest(['/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_extra.h5'], ['/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz'], 
                            '/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M_norm_params/IWCDmPMT_4pi_fulltank_9M_trainval_norm_params.npz', 'identity', 'identity',
                               shuffle=True, num_datasets=1, test_subset=None, collapse_arrays=False)

Loading training indices from:  /fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz
Loading validation indices from:  /fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz
Loading test indices from:  /fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs_extra.npz


In [97]:
dsets = [train_dset, val_dset, test_dset]
idxs = [np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)[idx_set_name] for idx_set_name in ['train_idxs', 'val_idxs', 'test_idxs']]
for d_idx, dset in enumerate(dsets):
    pbar = ProgressBar(widgets=['Checking For Charge Sum: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(dset))
    pbar.start()
    for i, idx in enumerate(range(len(idxs[j]))):
        assert np.sum(dset[idxs[j][i]][0])
        pbar.update(i)
    pbar.finish()

Checking For Charge Sum: 100% [000000000000000000000000000000000] Time: 0:14:08


AttributeError: 'list' object has no attribute 'shape'

In [88]:
import numpy as np
import h5py
from progressbar import *
h5file = h5py.File("/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5","r")
idxfile = np.load("/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz")
pbar = ProgressBar(widgets=['Checking For Charge Sum: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),' ', ETA()], maxval=100000)
pbar.start()
for i in idxfile['train_idxs'][:100000]:
    totq = sum(h5file['hit_charge'][h5file['event_hits_index'][i]:h5file['event_hits_index'][i+1]])
    assert totq > 300, totq
    pbar.update(pbar.currval+1)
pbar.finish()

KeyboardInterrupt: 

In [76]:
sum(h5file['hit_charge'][h5file['event_hits_index'][0]:h5file['event_hits_index'][1]])

2928.957929134369